<a href="https://colab.research.google.com/github/wiederm/notebooks/blob/main/UmbrellaPMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()